In [1]:
import requests
import random
import json

base_url = "http://localhost:8081"

resp = requests.get(base_url)
assert resp.status_code == 404
assert resp.json()["status"] == 404
assert resp.json()["error"] == "Not Found"


In [2]:

resp = requests.get("%s/api/docs/swagger.json"%base_url)
assert resp.status_code == 200
assert resp.json()["swagger"] == "2.0"
assert resp.json()["paths"] != None

swaggerJson = resp.json()
swaggerJson["paths"].keys()

jsonUrl = [f for f in swaggerJson["paths"].keys() if ".json" in f][8]
jsonUrl

resp = requests.get("%s%s"%(base_url, jsonUrl))
assert resp.status_code == 200
resp.json()

{'Institute': {'$id': '#/properties/Institute',
  'type': 'object',
  'title': 'Instituteschema',
  'required': ['name', 'username'],
  'uniqueIndexFields': ['username'],
  'properties': {'name': {'type': 'string'},
   'phoneNumber': {'type': 'string'},
   'email': {'type': 'string'},
   'username': {'type': 'string'},
   'address': {'type': 'string'}}}}

In [3]:
entities = list(resp.json().keys())
print("Available entities ", entities)

Available entities  ['Institute']


In [4]:
entity_name=entities[0]
print("Using entity %s"%entity_name)


Using entity Institute


In [5]:
userId =str(random.randint(1e10,1e11))
resp = requests.post("%s%s"%(base_url, '/api/v1/SkillCertificate'), json={
    "name":"KM GURIYA", 
    "contact": userId, 
    "rollNumber": "224300003",
    "dob": "1997-09-20",
    "gender": "F",
    "email": "kumarpawans@gmail.com",
    "trainingTitle":"DIPLOMA IN GENERAL NURSING and MIDWIFERY",
    "enrollmentNumber":"2210254300051",
    "examYear":"2022"
})
print(resp)
#assert resp.status_code == 200 or print (resp.json())
#idx = resp.json()["result"][entity_name]["osid"]


<Response [200]>


In [6]:
resp.json()


{'id': 'sunbird-rc.registry.create',
 'ver': '1.0',
 'ets': 1688041844079,
 'params': {'resmsgid': '',
  'msgid': '4336836b-119f-451b-b803-596cdf5413ec',
  'err': '',
  'status': 'SUCCESSFUL',
  'errmsg': ''},
 'responseCode': 'OK',
 'result': {'SkillCertificate': {'osid': '1-5130793b-36b6-40fb-829e-40d96ef8a138'}}}

In [7]:
print(resp.json())
resp.status_code, userId


{'id': 'sunbird-rc.registry.create', 'ver': '1.0', 'ets': 1688041844079, 'params': {'resmsgid': '', 'msgid': '4336836b-119f-451b-b803-596cdf5413ec', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'SkillCertificate': {'osid': '1-5130793b-36b6-40fb-829e-40d96ef8a138'}}}


(200, '55836041326')

In [8]:
print(entity_name)
print("120fbb69-db3b-4ae1-b00c-3817246d6fe2")
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, "1-a9029131-5b37-45c8-a2df-6e4b244f8172"), headers={"Accept":"application/json"})
print(json.dumps(resp.json()))
#print(resp.json()["proof"]["type"])
#assert resp.json()["proof"]["type"] == "Ed25519Signature2018"


Institute
120fbb69-db3b-4ae1-b00c-3817246d6fe2


{"id": "sunbird-rc.registry.read", "ver": "1.0", "ets": 1688041884402, "params": {"resmsgid": "", "msgid": "065937d8-1738-47a6-85b1-139ac8589064", "err": "", "status": "UNSUCCESSFUL", "errmsg": "Record with a9029131-5b37-45c8-a2df-6e4b244f8172 not found in Elastic-search"}, "responseCode": "OK"}


In [9]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, "1-a9029131-5b37-45c8-a2df-6e4b244f8172"), headers={"Accept":"application/pdf"})


In [10]:
resp.status_code, resp.content

assert resp.content[:5].decode().startswith("%PDF")
with open('sample2.pdf', 'wb') as f:
    f.write(resp.content)
    

AssertionError: 

In [ ]:
%system open 'sample2.pdf'


["'open' is not recognized as an internal or external command,",
 'operable program or batch file.']

In [ ]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"text/html"})
resp.status_code, resp.content

assert resp.content[:5].decode().startswith("<html")
with open('sample.html', 'wb') as f:
    f.write(resp.content)
%system open 'sample.html'


AssertionError: 

In [ ]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"image/svg+xml"})
resp.status_code, resp.content

assert resp.content.decode().find("<svg") != -1
with open('sample.svg', 'wb') as f:
    f.write(resp.content)
%system open 'sample.svg'

[]

# generate certificate by external templates

In [ ]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/pdf", "template": "https://raw.githubusercontent.com/dileepbapat/ref-sunbirdrc-certificate/main/schemas/templates/TrainingCertificate.html"})
resp.status_code, resp.content

assert resp.content[:5].decode().startswith("%PDF")
with open('sample_external222.pdf', 'wb') as f:
    f.write(resp.content)
%system open 'sample_external222.pdf'


["'open' is not recognized as an internal or external command,",
 'operable program or batch file.']

# delete certificate

In [ ]:
esp = requests.delete("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/json"})
print(resp.status_code)
assert resp.status_code == 200
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/json"})
print(json.dumps(resp.json()))
assert resp.json()["params"]["errmsg"] == "entity status is inactive"


200
{"id": "sunbird-rc.registry.read", "ver": "1.0", "ets": 1646624071473, "params": {"resmsgid": "", "msgid": "45967509-87eb-4dc4-997d-777c251dbc00", "err": "", "status": "UNSUCCESSFUL", "errmsg": "entity status is inactive"}, "responseCode": "OK"}


# generate credentials with external template

In [ ]:
userId =str(random.randint(1e10,1e11))
resp = requests.post("%s%s"%(base_url, '/api/v1/SkillCertificate'), json={
    "name":"Sunbird Learner", 
    "contact": userId, 
    "trainingTitle":"Sunbird RC Certificate Module",
    "skills": [
        {
            "skill": "XXXXX",
            "certifiedOn": "12-12-2000"
        },
        {
            "skill": "network training",
            "certifiedOn": "12-11-2000"
        }

    ]
    
})
assert resp.status_code == 200 or print (resp.json())
idx = resp.json()["result"]["SkillCertificate"]["osid"]
print(resp.json())

{'id': 'sunbird-rc.registry.create', 'ver': '1.0', 'ets': 1683524652746, 'params': {'resmsgid': '', 'msgid': 'f2d7d79d-ac30-4620-8d80-49f98cfb2162', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'SkillCertificate': {'osid': '1-3be10c00-32ce-4e90-b340-af1aaf0f1e2b'}}}


In [ ]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, 'SkillCertificate', idx), headers={"Accept":"application/vc+ld+json"})
print(json.dumps(resp.json()))
assert resp.json()["proof"]["type"] == "RsaSignature2018"


{"@context": ["https://www.w3.org/2018/credentials/v1", {"@context": {"@version": 1.1, "@protected": true, "SkillCertificate": {"@id": "https://github.com/sunbird-specs/vc-specs#SkillCertificate", "@context": {"id": "@id", "@version": 1.1, "@protected": true, "skills": "schema:Text"}}, "Person": {"@id": "https://github.com/sunbird-specs/vc-specs#Person", "@context": {"name": "schema:Text"}}, "trainedOn": {"@id": "https://github.com/sunbird-specs/vc-specs#trainedOn", "@context": {"name": "schema:Text"}}, "skills": {"@id": "https://github.com/sunbird-specs/vc-specs#skills", "@container": "@list"}}}], "type": ["VerifiableCredential"], "issuanceDate": "2021-08-27T10:57:57.237Z", "credentialSubject": {"type": "Person", "name": "Sunbird Learner", "trainedOn": "Sunbird RC Certificate Module", "skills": ["XXXXX", "network training", ""]}, "issuer": "did:web:sunbirdrc.dev/vc/skill", "proof": {"type": "RsaSignature2018", "created": "2023-05-08T05:44:13Z", "verificationMethod": "did:india", "proo

In [ ]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, 'SkillCertificate', idx), headers={"Accept":"application/pdf", "template": "http://localhost:63342/sunbird-rc/registry/public/_schemas/templates/SkillCertificate.html?_ijt=qu4536mc0j899963igug2gitf0"})

resp.status_code, resp.content
print(resp)
assert resp.content[:5].decode().startswith("%PDF")
with open('sample122.pdf', 'wb') as f:
    f.write(resp.content)
    
%system open 'sample122.pdf'


<Response [200]>


AssertionError: 